In [1]:
import numpy as np
from qiskit import *
from qiskit.quantum_info import Operator, Pauli, SparsePauliOp
from tabulate import tabulate
from itertools import combinations
from scipy.linalg import eigh, block_diag
import math,cmath

In [2]:
def spinless_sub_dim(N,r) : 
    '''
    input : number of lattice sites (N), number of electrons (r) 
    output : dimension of this subspace 
    '''
    return math.comb(N,r)

def spinless_basis(N,r) : 
    basis_set = []
    lattice = list(range(N))
    places = list(combinations(lattice, r))
    for combination in places : 
        basis = [False] *N
        for index in combination : 
            basis[index] = True 
        basis_set.append(basis)
    return basis_set

def spinless_states_index(N) : 
    '''Input : N, uses spinless_basis function
    Output : Returns binary of basis states in fermionic Hamiltonian ''' 
    index = []
    for r in range(N+1) : 
        b_set = spinless_basis(N,r)
        new_bset = []
        for b in b_set : 
            new_b = sum([2**(N-i-1)*b[i] for i in range(N)])
            index.append(new_b)
    return index

def tb0_JW(N,e,t) : 
    strings = []
    opt = SparsePauliOp.from_sparse_list([("I", [0], 0)], num_qubits=N)  
    for k in range(N) :
        a0='I'*(N)
        a1 = 'I'*(k)+'Z' +'I'*(N-k-1)

        b0='I'*N
        b0_list = list(b0)
        b0_list[k] = 'X'
        b0_list[(k+1)%N] = 'X'
        new_b0 = ''.join(b0_list)

        b1='I'*N
        b1_list = list(b0)
        b1_list[k] = 'Y'
        b1_list[(k+1)%N] = 'Y'
        new_b1 = ''.join(b1_list)

        strings.append([a0,a1,new_b0,new_b1])
        val = 1
        if N==2 and k==1 : 
            val = 0
        opt += SparsePauliOp.from_list([(a0, 0.5*e[k]), (a1, -0.5*e[k]),(new_b0, 0.5*t*val),(new_b1, 0.5*t*val)])
    return opt  

def ferm_JW(JW_mat) : 
    '''Input : SparsePauliOp JW matrix, uses function spinless_states_index
    Output : Returns JW matrix in order of fermionic basis states'''
    class_index = spinless_states_index(N)
    #JW_mat = mat.to_matrix()
    ferm_mat = np.zeros((2**N,2**N))
    for i in range(2**N) : 
        for j in range(2**N) : 
            ferm_mat[i][j] = JW_mat[class_index[i]][class_index[j]]
    return ferm_mat

def tb0_model(N,r,e,t): 
    ''' Generalised Tight Binding (spinless) model for periodic boundary condition
    Input : Number of lattice sites (int), number of electrons(int), hopping constant (int/float), onsite energies (list)
    Output : Tight binding Hamiltonian, eigenvalues and eigenvectors of the matrix ''' 
    dim = spinless_sub_dim(N,r)
    if r==0 : 
        H = np.zeros(1)
        eigval = H[0]
        new_vec = H
    elif r==N : 
        H = [[r]]
        eigval = H[0]
        new_vec = H
    else : 
        H = np.zeros((dim, dim))
        basis_set = spinless_basis(N,r)
        n_diag = np.zeros(dim)
        for i in range(dim) : 
            for j in range(N) : 
                n_diag[i] += e[j]*basis_set[i][j]

        np.fill_diagonal(H,n_diag)
        for basis_index,basis in enumerate(basis_set) : 
            for site in range(len(basis)) : 
                if basis[site] == False and basis[(site+1)%N] == True : 
                    new_state = basis.copy()
                    new_state[site] = True
                    new_state[(site+1)%N] = False 
                    for i in range(len(basis_set)) : 
                        if basis_set[i] == new_state: 
                            f_index = i
                    H[f_index][basis_index] +=t
                if N != 2 : 
                    if basis[site] == True and basis[(site+1)%N] == False : 
                        new_state = basis.copy()
                        new_state[site] = False
                        new_state[(site+1)%N] = True 
                        for i in range(len(basis_set)) : 
                            if basis_set[i] == new_state : 
                                f_index = i
                        H[f_index][basis_index] +=t 
        eigval,eigvec = np.linalg.eigh(H)
        new_vec = list(zip(*eigvec))
                
    return H,eigval,new_vec

def tb0_full(N,e,t): 
    '''Full Block Diagonal Hamiltonian for some N length closed lattice '''
    H_sub_list = []
    for r in range(N+1) : 
        H_sub = tb0_model(N,r,e,t)[0]
        H_sub_list.append(H_sub)
    H = block_diag(*H_sub_list) 
    return H

In [26]:
N = 10
e = [1]*N
t = 7

In [27]:
eig_jw,v_jw = np.linalg.eigh(tb0_JW(N,e,t))
vec_jw = list(zip(*v_jw))

In [29]:
print(eig_jw)

# print('=========')

# for vec in vec_jw : 
#     print(list(vec))
#     print('---------')

[-40.30495168 -39.08756952 -37.08756952 ...  47.08756952  49.08756952
  50.30495168]


In [25]:
for r in range(N+1) : 
    print(spinless_sub_dim(N,r))

1
5
10
10
5
1


In [14]:
circ = QuantumCircuit(N)
#circ.initialize('1101',circ.qubits)
time=1e-10
JW_op = tb0_JW(N,e,t)
circ.hamiltonian(JW_op,time,list(range(N)))
#circ.decompose(reps=2).draw('mpl')

In [15]:
backend = Aer.get_backend('unitary_simulator')
job = execute(circ, backend, shots=1000)
result = job.result()


U = result.get_unitary(circ,10).data
print(U)
# #print(tabulate(U,tablefmt='plain'))
# # H_circ = (U-np.eye(2**N))*(1j/time)
# # #H = (U-np.eye(2**N))
# # #print(H_circ)
# # print(tabulate(H_circ,tablefmt='plain'))
# # print('------')
# # H_circ_ferm = ferm_JW(H_circ)
# # print(tabulate(H_circ_ferm,tablefmt='plain'))

[[ 1.+0.e+00j  0.+0.e+00j  0.+0.e+00j  0.+0.e+00j  0.+0.e+00j  0.+0.e+00j
   0.+0.e+00j  0.+0.e+00j]
 [ 0.+0.e+00j  1.-1.e-10j -0.-7.e-10j  0.+0.e+00j -0.-7.e-10j  0.+0.e+00j
   0.+0.e+00j  0.+0.e+00j]
 [ 0.+0.e+00j -0.-7.e-10j  1.-1.e-10j  0.+0.e+00j -0.-7.e-10j  0.+0.e+00j
   0.+0.e+00j  0.+0.e+00j]
 [ 0.+0.e+00j  0.+0.e+00j  0.+0.e+00j  1.-2.e-10j  0.+0.e+00j -0.-7.e-10j
  -0.-7.e-10j  0.+0.e+00j]
 [ 0.+0.e+00j -0.-7.e-10j -0.-7.e-10j  0.+0.e+00j  1.-1.e-10j  0.+0.e+00j
   0.+0.e+00j  0.+0.e+00j]
 [ 0.+0.e+00j  0.+0.e+00j  0.+0.e+00j -0.-7.e-10j  0.+0.e+00j  1.-2.e-10j
  -0.-7.e-10j  0.+0.e+00j]
 [ 0.+0.e+00j  0.+0.e+00j  0.+0.e+00j -0.-7.e-10j  0.+0.e+00j -0.-7.e-10j
   1.-2.e-10j  0.+0.e+00j]
 [ 0.+0.e+00j  0.+0.e+00j  0.+0.e+00j  0.+0.e+00j  0.+0.e+00j  0.+0.e+00j
   0.+0.e+00j  1.-3.e-10j]]


In [16]:
eig_U,v_U = np.linalg.eig(U)
vec_U = list(zip(*v_U))

In [18]:
print(eig_U)
print(min(eig_U))
eigUnew = [1j/time*cmath.log(x) for x in eig_U]
#print(eigUnew)
U_list = []
for vec in vec_U : 
    
    new_vec = [np.abs(x) for x in vec]
    U_list.append(new_vec)
    print(new_vec)
    print('------------------')

[1.+5.00000041e-10j 1.-1.60000002e-09j 1.+6.00000161e-10j
 1.-1.49999993e-09j 1.+6.00000161e-10j 1.+5.00000048e-10j
 1.+0.00000000e+00j 1.-3.00000000e-10j]
(0.9999999999999996-1.4999999298215272e-09j)
[0.0, 0.0, 0.0, 0.4082482904638632, 0.0, 0.4082482904638632, 0.8164965809277257, 0.0]
------------------
[0.0, 0.0, 0.0, 0.5773502893018719, 0.0, 0.5773502893018719, 0.5773502289651312, 0.0]
------------------
[0.0, 0.816496580927667, 0.4082482904638428, 1.6481677288337601e-07, 0.4082482904638329, 1.6481677288337511e-07, 2.8842930810029037e-07, 0.0]
------------------
[0.0, 0.5773502589065944, 0.5773502743310891, 2.0832076613016943e-07, 0.577350274331075, 2.0832076613017065e-07, 2.2434543168315403e-07, 0.0]
------------------
[0.0, 0.28224668497442107, 0.5223919976637438, 1.8683011157668623e-07, 0.8046386826382277, 7.363292508371665e-08, 9.970428840619338e-08, 0.0]
------------------
[0.0, 8.665884825279275e-07, 1.3929815610481225e-06, 0.6111706723789928, 6.612699559025508e-07, 0.77446997

In [150]:
# indices = spinless_states_index(N)
# jordan_list = []
# for vec in vec_U : 
#     ferm_vec = [0]*len(vec)
#     for i in range(len(vec)) : 
#         ferm_vec[i] = float(vec[indices[i]])
#     print(list(ferm_vec))
#     jordan_list.append(list(ferm_vec))
#     print('---------')

In [151]:
# backend = Aer.get_backend('statevector_simulator')
# job = execute(circ, backend)
# result = job.result()
# statevector = result.get_statevector()

# print(statevector)


In [25]:
eigfull,vecfull = np.linalg.eigh(tb0_full(N,e,t))
eigvecfull = list(zip(*vecfull))

In [22]:
eig_jw,v_jw = np.linalg.eigh(tb0_JW(N,e,t))
vec_jw = list(zip(*v_jw))

In [26]:
print(eigfull)

[-6. -6. -5. -5.  0.  3. 15. 16.]


In [23]:
print(eig_jw)

[-6. -6. -5. -5.  0.  3. 15. 16.]


In [27]:
normal_list = []
for vec in eigvecfull : 
    print(list(vec))
    normal_list.append(list(vec))
    print('---------')

[0.0, 0.0, -0.7071067811865476, 0.7071067811865475, 0.0, 0.0, 0.0, 0.0]
---------
[-0.0, -0.8164965809277261, 0.40824829046386313, 0.408248290463863, -0.0, -0.0, -0.0, -0.0]
---------
[0.0, 0.0, 0.0, 0.0, 0.0, -0.7071067811865476, 0.7071067811865475, 0.0]
---------
[-0.0, -0.0, -0.0, -0.0, -0.8164965809277261, 0.40824829046386313, 0.408248290463863, -0.0]
---------
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---------
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
---------
[0.0, -0.5773502691896257, -0.5773502691896261, -0.577350269189626, 0.0, 0.0, 0.0, 0.0]
---------
[0.0, 0.0, 0.0, 0.0, -0.5773502691896257, -0.5773502691896261, -0.577350269189626, 0.0]
---------


In [24]:
indices = spinless_states_index(N)
jordan_list = []
for vec in vec_jw : 
    ferm_vec = [0]*len(vec)
    for i in range(len(vec)) : 
        ferm_vec[i] = float(vec[indices[i]])
    print(list(ferm_vec))
    jordan_list.append(list(ferm_vec))
    print('---------')

[0.0, -0.7071067811865475, 0.7071067811865476, 0.0, 0.0, 0.0, 0.0, 0.0]
---------
[-0.0, 0.408248290463863, 0.40824829046386313, -0.8164965809277261, 0.0, 0.0, 0.0, -0.0]
---------
[0.0, 0.0, 0.0, 0.0, 0.7071067811865475, -0.7071067811865476, 0.0, 0.0]
---------
[0.0, 0.0, 0.0, 0.0, -0.408248290463863, -0.40824829046386313, 0.8164965809277261, 0.0]
---------
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---------
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
---------
[0.0, -0.577350269189626, -0.5773502691896261, -0.5773502691896257, 0.0, 0.0, 0.0, 0.0]
---------
[0.0, 0.0, 0.0, 0.0, -0.577350269189626, -0.5773502691896261, -0.5773502691896257, 0.0]
---------


C:\Users\cheshtajoshi\AppData\Local\Temp\ipykernel_7848\2005635066.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  ferm_vec[i] = float(vec[indices[i]])


In [158]:
for i in range(len(normal_list)) : 
    #print(max(normal_list[i]-jordan_list[i]))
    result = []
    for j in range(len(normal_list)):
        result.append(normal_list[i][j] - jordan_list[i][j])
    print(sum(result))

0.0
0.0
0.0
0.0


In [159]:
for i in range(len(normal_list)) : 
    #print(max(normal_list[i]-jordan_list[i]))
    result = []
    for j in range(len(normal_list)):
        result.append(U_list[i][j] - jordan_list[i][j])
    print((result))

[0.0, 4.440892098500626e-16, 1.414213562373095, 0.0]
[-1.0, 0.7071067811865477, 0.7071067811865475, 0.0]
[1.0, 0.0, 0.0, -1.0]
[0.0, -0.7071067811865475, -0.7071067811865475, 1.0]


In [160]:
H_JW = tb0_JW(N,e,t).to_matrix()

In [161]:
print(tabulate(H_JW,tablefmt='plain'))

0  0  0  0
0  1  7  0
0  7  1  0
0  0  0  2


In [ ]:
for r in range(N+1) : 
    print(spinless_basis(N,r))
    print('----')
    

In [ ]:
spinless_states_index(N)

In [ ]:
tb0_JW(N,e,t).to_matrix()